In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
box = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 1\boxes.csv')
purchase = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 1\purchase.csv')

In [3]:
box.isnull().sum()

BOX_ID             0
QUALITY            0
DELIVERY_OPTION    0
MILK               0
MEAT               0
UNIT_PRICE         0
dtype: int64

In [4]:
purchase.duplicated().sum()

77

In [5]:
box.duplicated().sum()

0

In [6]:
purchase.isnull().sum()

PURCHASE_DATE     0
MAGIC_KEY         0
BOX_ID           47
BOX_COUNT        47
dtype: int64

In [7]:
purchase.dropna(subset=['BOX_ID', 'BOX_COUNT'], inplace=True)

# Convert PURCHASE_DATE to datetime
purchase['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], format='%d/%m/%Y')

# Convert BOX_ID to integer
purchase['BOX_ID'] = purchase['BOX_ID'].astype('int32')

# Convert BOX_COUNT to integer
purchase['BOX_COUNT'] = purchase['BOX_COUNT'].astype('int32')

box['BOX_ID'] = box['BOX_ID'].astype('int32')

box = box.drop_duplicates()
purchase = purchase.drop_duplicates()

In [8]:
df = pd.merge(box, purchase, how='inner')
df.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,2019-02-04,2C88D36D1FC,1
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2BF011BDB38,1
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA0EE8F2B3,1
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2C623730B09,1
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA6CE1054F,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455723 entries, 0 to 2455722
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   BOX_ID           int32         
 1   QUALITY          object        
 2   DELIVERY_OPTION  object        
 3   MILK             float64       
 4   MEAT             float64       
 5   UNIT_PRICE       float64       
 6   PURCHASE_DATE    datetime64[ns]
 7   MAGIC_KEY        object        
 8   BOX_COUNT        int32         
dtypes: datetime64[ns](1), float64(3), int32(2), object(3)
memory usage: 149.9+ MB


In [11]:
# Filter the data for February 2019
filtered_df = df[(df['PURCHASE_DATE'] >= '2019-02-01') & (df['PURCHASE_DATE'] <= '2019-02-28')]

# Calculate the sum of the 'MEAT' column
total_meat_sales = filtered_df['MEAT'].sum()

In [12]:
print(total_meat_sales)

1269193.2999999996


In [13]:
# Calculate the total sales amount per transaction
df['MEAT_SALES_AMOUNT'] = df['MEAT'] * df['UNIT_PRICE']

# Group by month and sum the sales amounts
monthly_sales = df.resample('M', on='PURCHASE_DATE')['MEAT_SALES_AMOUNT'].sum()

# Calculate the month-on-month growth rates
monthly_growth = monthly_sales.pct_change()

# Average of these growth rates
average_growth_percentage = monthly_growth.mean() * 100  # Convert to percentage

In [14]:
print(average_growth_percentage)

10.066026501004755


In [15]:
# Filter for January 2019
jan_19_df = df[(df['PURCHASE_DATE'] >= '2019-01-01') & (df['PURCHASE_DATE'] <= '2019-01-31')]

# Calculate the milk sales amount per transaction
jan_19_df['MILK_SALES_AMOUNT'] = jan_19_df['MILK'] * jan_19_df['UNIT_PRICE']

# Group by day of the week (0=Monday, 6=Sunday)
daily_avg_milk_sales = jan_19_df.groupby(jan_19_df['PURCHASE_DATE'].dt.dayofweek)['MILK_SALES_AMOUNT'].mean()

# Get the averages for Sunday (6) and Tuesday (2)
avg_sunday_sales = daily_avg_milk_sales.get(6, 0)  # Default to 0 if no sales
avg_tuesday_sales = daily_avg_milk_sales.get(2, 0)  # Default to 0 if no sales

# Calculate the difference
difference = avg_sunday_sales - avg_tuesday_sales

C:\Users\Admin\AppData\Local\Temp\ipykernel_18212\4102417941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_19_df['MILK_SALES_AMOUNT'] = jan_19_df['MILK'] * jan_19_df['UNIT_PRICE']


In [16]:
print(difference)

-0.046212941855998224


In [17]:
# Filter for November 2018 and quality 'Premium'
nov_18_premium_df = df[(df['PURCHASE_DATE'] >= '2018-11-01') & 
                       (df['PURCHASE_DATE'] <= '2018-11-30') &
                       (df['QUALITY'] == 'Premium')]

# Group by BOX_ID and sum BOX_COUNT
box_sales = nov_18_premium_df.groupby('BOX_ID')['BOX_COUNT'].sum()

# Find the BOX_ID with the highest sales
top_selling_box_id = box_sales.idxmax()
top_selling_sales_count = box_sales.max()

In [18]:
print(top_selling_box_id)

163


In [19]:
# Filter for December 2018
dec_18_df = df[(df['PURCHASE_DATE'] >= '2018-12-01') & (df['PURCHASE_DATE'] <= '2018-12-31')]

# Group by BOX_ID and sum BOX_COUNT
box_sales = dec_18_df.groupby('BOX_ID')['BOX_COUNT'].sum()

# Sort the sales descending and get the index of the third highest
third_highest_box_id = box_sales.sort_values(ascending=False).index[2]  # Indexing is zero-based

# Get the unit price for the third highest sold box
# Assuming unit price is constant for each box id across the dataset
third_highest_unit_price = dec_18_df[dec_18_df['BOX_ID'] == third_highest_box_id]['UNIT_PRICE'].iloc[0]

In [20]:
print(third_highest_unit_price)

15.96
